# Air Quality Data Preparation

This notebook:

1. Combines daily AQI data for California counties (2020–2024)
2. Aggregates daily AQI to weekly averages by county
3. Applies 2020 Census county population weights
4. Produces a population-weighted weekly statewide AQI series

The final dataset will align with weekly NHSN respiratory admissions data.

In [5]:
import pandas as pd
import numpy as np
import glob

## Step 1: Combine 2020–2024 Daily AQI Files

Each file contains:
- County Name
- Date
- AQI
- Number of Sites Reporting

We stack all years into one master dataset.

In [6]:
# If files are uploaded manually in Colab
file_paths = [
    "daily_aqi_by_CAcounty_2020.xlsx",
    "daily_aqi_by_CAcounty_2021.xlsx",
    "daily_aqi_by_CAcounty_2022.xlsx",
    "daily_aqi_by_CAcounty_2023.xlsx",
    "daily_aqi_by_CAcounty_2024.xlsx"
]

# Read and concatenate
df_list = [pd.read_excel(file) for file in file_paths]
aqi_daily = pd.concat(df_list, ignore_index=True)

# Keep relevant columns only
aqi_daily = aqi_daily[[
    "county Name",
    "Date",
    "AQI"
]]

# Convert date column
aqi_daily["Date"] = pd.to_datetime(aqi_daily["Date"])

# Sort
aqi_daily = aqi_daily.sort_values(["county Name", "Date"])

aqi_daily.head()

,county Name,Date,AQI
0,Alameda,2020-01-01,58
1,Alameda,2020-01-02,53
2,Alameda,2020-01-03,77
3,Alameda,2020-01-04,54
4,Alameda,2020-01-05,43


## Step 2: Aggregate Daily AQI to Weekly (By County)

We:
- Convert daily dates to week-ending dates
- Compute weekly mean AQI for each county

This matches the weekly structure of the hospital admissions dataset.

In [8]:
# Ensure Date is datetime (coerce errors to NaT)
aqi_daily["Date"] = pd.to_datetime(aqi_daily["Date"], errors="coerce")

# Drop rows where Date is missing
aqi_daily = aqi_daily.dropna(subset=["Date"])

# Create week-ending Sunday date (cleaner + no lambda needed)
aqi_daily["Week"] = (
    aqi_daily["Date"]
    .dt.to_period("W-SUN")
    .dt.end_time
)

# Aggregate to weekly mean AQI by county
aqi_weekly_county = (
    aqi_daily
    .groupby(["county Name", "Week"], as_index=False)["AQI"]
    .mean()
)

aqi_weekly_county.head()

,county Name,Week,AQI
0,Alameda,2020-01-05 23:59:59.999999999,57.000000
1,Alameda,2020-01-12 23:59:59.999999999,45.857143
2,Alameda,2020-01-19 23:59:59.999999999,44.142857
3,Alameda,2020-01-26 23:59:59.999999999,50.000000
4,Alameda,2020-02-02 23:59:59.999999999,54.714286


## Step 3: Apply 2020 Census County Population Weights

We use 2020 Census population estimates for each California county.

Population weighting ensures that counties with larger populations
contribute proportionally more to the statewide AQI estimate.

In [10]:
# 2020 U.S. Census County Populations (California)

county_population = {
    "Alameda": 1680614,
    "Alpine": 1179,
    "Amador": 40584,
    "Butte": 210222,
    "Calaveras": 45350,
    "Colusa": 21868,
    "Contra Costa": 1166165,
    "Del Norte": 27616,
    "El Dorado": 191250,
    "Fresno": 1009613,
    "Glenn": 28889,
    "Humboldt": 136324,
    "Imperial": 179730,
    "Inyo": 18999,
    "Kern": 906124,
    "Kings": 152797,
    "Lake": 68200,
    "Lassen": 32319,
    "Los Angeles": 9996634,
    "Madera": 156434,
    "Marin": 261341,
    "Mariposa": 17133,
    "Mendocino": 91362,
    "Merced": 282963,
    "Modoc": 8677,
    "Mono": 13225,
    "Monterey": 440506,
    "Napa": 137465,
    "Nevada": 102267,
    "Orange": 3186261,
    "Placer": 405951,
    "Plumas": 19761,
    "Riverside": 2424663,
    "Sacramento": 1586727,
    "San Benito": 64533,
    "San Bernardino": 2183526,
    "San Diego": 3301182,
    "San Francisco": 874826,
    "San Joaquin": 780676,
    "San Luis Obispo": 281924,
    "San Mateo": 762727,
    "Santa Barbara": 448083,
    "Santa Clara": 1924802,
    "Santa Cruz": 271768,
    "Shasta": 182190,
    "Sierra": 3229,
    "Siskiyou": 43999,
    "Solano": 452887,
    "Sonoma": 488501,
    "Stanislaus": 553710,
    "Sutter": 99479,
    "Tehama": 65690,
    "Trinity": 16095,
    "Tulare": 473944,
    "Tuolumne": 55397,
    "Ventura": 843808,
    "Yolo": 217768,
    "Yuba": 82001
}

# Convert dictionary to dataframe
pop_df = pd.DataFrame(
    list(county_population.items()),
    columns=["county Name", "Population"]
)

# Clean county names just in case (prevents merge errors)
aqi_weekly_county["county Name"] = aqi_weekly_county["county Name"].str.strip()
pop_df["county Name"] = pop_df["county Name"].str.strip()

# Merge population into weekly county AQI
aqi_weekly_county = aqi_weekly_county.merge(
    pop_df,
    on="county Name",
    how="left"
)

# Check for missing population matches
missing_counties = aqi_weekly_county[aqi_weekly_county["Population"].isna()]["county Name"].unique()

if len(missing_counties) > 0:
    print("⚠️ Missing population for:", missing_counties)
else:
    print("✅ All counties successfully matched with population data.")

aqi_weekly_county.head()

✅ All counties successfully matched with population data.


,county Name,Week,AQI,Population
0,Alameda,2020-01-05 23:59:59.999999999,57.000000,1680614
1,Alameda,2020-01-12 23:59:59.999999999,45.857143,1680614
2,Alameda,2020-01-19 23:59:59.999999999,44.142857,1680614
3,Alameda,2020-01-26 23:59:59.999999999,50.000000,1680614
4,Alameda,2020-02-02 23:59:59.999999999,54.714286,1680614


## Step 4: Compute Weekly Population-Weighted Statewide AQI

Formula:

Weighted AQI =
    Σ (County Weekly AQI × County Population)
    ------------------------------------------------
    Σ (County Population)

This produces a single statewide AQI value per week.

In [11]:
# Multiply AQI by population
aqi_weekly_county["Weighted_AQI"] = (
    aqi_weekly_county["AQI"] * aqi_weekly_county["Population"]
)

# Compute statewide weekly weighted AQI
aqi_weekly_state = (
    aqi_weekly_county
    .groupby("Week")
    .apply(lambda x: x["Weighted_AQI"].sum() / x["Population"].sum())
    .reset_index(name="Statewide_Weighted_AQI")
)

aqi_weekly_state.head()

/tmp/ipython-input-1795/2741667168.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x["Weighted_AQI"].sum() / x["Population"].sum())


,Week,Statewide_Weighted_AQI
0,2020-01-05 23:59:59.999999999,73.010343
1,2020-01-12 23:59:59.999999999,56.445761
2,2020-01-19 23:59:59.999999999,63.021082
3,2020-01-26 23:59:59.999999999,64.843702
4,2020-02-02 23:59:59.999999999,59.804260


## Final Output

`aqi_weekly_state` now contains:

- Week
- Statewide_Weighted_AQI

This dataset can now be merged with weekly respiratory hospital admissions
using the week-ending date as the key.

In [12]:
aqi_weekly_state.to_csv("weekly_weighted_CA_AQI_2020_2024.csv", index=False)